 <a name="Overview"></a>
# Overview

[Global Imports](#GlobImp)

[Global Classes](#GlobClas)

[Global Variables and Functions](#GlobVarFunc)

[Mathematical Functions](#MathFunc)

[Graphical Output](#GraphOut)

[Interactive Elements](#IntElem)

[Layout](#Layout)

<a name="GlobImp"></a>
# Global Imports

In [22]:
%matplotlib inline
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import bqplot as bqp
import ipywidgets as widgets
plt.style.use('ggplot')
sp.init_printing()


 <a name="GlobClas"></a>
# Global Classes 
[Overview](#Overview)

In [36]:
# graphing classes

# line class
class is_line:
    def __init__(self, x_data, y_data, expr, x_sc, y_sc):
        self.x_data = x_data
        self.y_data = y_data
        self.x_sc = x_sc
        self.y_sc = y_sc
        self.expr = expr
            
# graph class
class is_graph:
    active = False
    position = 3
    x_min=-1
    x_max=1
    x_log=False
    y_log=False
    fig=False
    x_steps = 100
    def __init__(self, x_label='X', y_label='Y', title_label='Title', \
                 x_min=x_min, x_max=x_max, x_log=x_log, y_log=y_log, x_steps=x_steps):
        self.x_label = x_label
        self.y_label = y_label        
        self.title_label = title_label
        self.x_min = x_min
        self.x_max = x_max
        self.x_log = x_log
        self.y_log = y_log
        selfx_steps = x_steps
        self.lines = []
        
    def addline(self, x_data, y_data, expr, x_sc, y_sc):
        self.lines.append(is_line(x_data, y_data))

<a name="GlobVarFunc"></a>
# Global Variables and Functions 
[Overview](#Overview)

In [48]:
#global variables
x, y, z, t = sp.symbols("x y z t")

expr = x
expr_str = str(expr)
expr_array = []
history_array = []
plot_array = []
fig_array = []
active_plot = 3
default_graph_settings = False
history_vbox = widgets.VBox(history_array)
viz_vbox = widgets.VBox(fig_array)
input_box = widgets.HBox([])
gui = widgets.HBox([])

#makes a numerical computation of sympy equation
def npfy(x_, x_data, expr_):
    f = sp.lambdify(x_, expr_, 'numpy')
    y_data = f(x_data)
    return y_data

# updates the gui
def update_gui():
    global gui
    gui = widgets.HBox([left_box, viz_vbox])
    display(gui)

# updates the graphs
def update_viz():
    global fig_array
    option_array = []
    for idx, plot in enumerate(plot_array):
        fig_array.append(plot.fig)
        option_array.append(idx)
    viz_vbox = widgets.VBox(fig_array)
    update_gui()

# shows the equation
def update_history():
    global expr_array
    global history_array
    global history_vbox
    history_array = []
    history_vbox.close()
    style = {'description_width': 'initial'}
    layout = widgets.Layout(display='inline-flex')
    option_array = []
    for idx, expr_ in enumerate(expr_array):
        equation = widgets.Label(value=r'\(' + sp.latex(expr_) + '\)', style=style, layout=layout)
        index = widgets.Label(value=str(idx))
        hexp_hbox = widgets.HBox([index, equation])
        history_array.append(hexp_hbox)
        option_array.append(idx)
    history_vbox = widgets.VBox(history_array)
    eq_dropdown.options = option_array
    update_gui()
    return history_array

#sets choosen expresion active
def setactivegraph(self):
    global active_plot
    active_plot = active_dropdown.value


#sets choosen expresion active
def setactive(self):
    global expr
    global expr_str
    pos = eq_dropdown.value
    expr = expr_array[pos]
    expr_str = str(expr)
    expr_input.value = expr_str
    

#appends expression array and updates output
def expr_array_append(expr0):
    global expr_array
    if len(expr_array) > 0:
        if expr_array[-1] is not expr0:
            expr_array.insert(0, expr0)
    else:
        expr_array.append(expr)
    update_history()


In [29]:
#reads and validates user input
def readexpr(change):
    global expr_str
    global expr
    global expr_array
    if expr_input.value is not '' and expr_input.value is not None:
        expr = sp.sympify(expr_input.value)
        expr_str = str(expr)
        eq_out0.value = r'\(' + sp.latex(expr) + '\)'
        expr_array_append(expr)
        return expr
        """
        try:
            expr = sp.sympify(expr_input.value)
            expr_str = str(expr)
            eq_out0.value = r'\(' + sp.latex(expr) + '\)'
            expr_array_append(expr)
            return expr
        except:
            eq_out0.value = r'Invalid Expression!'
            return 'Invalid Expression!'
        """


<a name="MathFunc"></a>
# Mathematical Functions 
[Overview](#Overview)

In [26]:
# simplify equation
def simplify(self):
    sexpr = sp.simplify(expr)
    eq_out1.value = r'\(' + sp.latex(sexpr) + '\)'
    expr_array_append(sexpr)

# expands equation
def expand(self):
    eexpr = sp.expand(expr)
    eq_out1.value = r'\(' + sp.latex(eexpr) + '\)'
    expr_array_append(eexpr)
        
# derivatives
def derivative(xyz):
    if xyz is 'x':                
        exprdif = sp.diff(expr, x)
    elif xyz is 'y':
        exprdif = sp.diff(expr, y)
    elif xyz is 'z':
        exprdif = sp.diff(expr, z)
    elif xyz is 't':
        exprdif = sp.diff(expr, t)
    else:
        exprdif = r'Invalid Differentition parameter!'
    eq_out1.value = r'\(' + sp.latex(exprdif) + '\)'
    expr_array_append(exprdif)

def derix(self):
    derivative('x')
    x
def deriy(self):
    derivative('y')
    
def deriz(self):
    derivative('z')
    
def derit(self):
    derivative('t')
    
# integrals
def integrate(self):
    iexpr = sp.integrate(expr)
    eq_out1.value = r'\(' + sp.latex(iexpr) + '\)'
    expr_array_append(iexpr)


<a name="GraphOut"></a>
# Graphical Output
[Overview](#Overview)

In [34]:
# adds a line to a given plot
def addline(plot, expr0):    
    if plot.x_log:
        x_data = np.geomspace(plot.x_min, plot.x_max, plot.x_steps)
    else:
        x_data =  np.linspace(plot.x_min, plot.x_max, plot.x_steps)
    y_data = npfy(x, x_data, expr0)
    x_sc = plot.fig.axes[0].scale
    y_sc = plot.fig.axes[1].scale
    newline = is_line(x_data, y_data, expr0, x_sc, y_sc)
    line = bqp.Lines(x=x_data, y=y_data, scales={'x': x_sc, 'y': y_sc})
    plot.lines.append(newline)
    plot.fig.marks.insert(0, line)
    return plot

# make a new plot
def makeplot():
    pos = eq_dropdown.value
    if default_graph_settings:
        dgs = default_graph_settings
        newplot = is_graph(dgs.x_label, dgs.y_label, dgs.title_label, \
                 dgs.x_min, dgs.x_max, dgs.x_log, dgs.y_log, dgs.x_steps)
    else:
        newplot = is_graph()
    if newplot.x_log:
        x_sc = bqp.LogScale(min=newplot.x_min, max=newplot.x_max)
    else:
        x_sc = bqp.LinearScale(min=newplot.x_min, max=newplot.x_max)
    if newplot.y_log:
        y_sc = bqp.LogScale(min=newplot.x_min, max=newplot.x_max)
    else:
        y_sc = bqp.LinearScale(min=newplot.x_min, max=newplot.x_max)
    ax_x = bqp.Axis(label=newplot.x_label, scale=x_sc, grid_lines='solid')
    ax_y = bqp.Axis(label=newplot.y_label, scale=y_sc, orientation='vertical', grid_lines='solid')
    newplot.fig = bqp.Figure(axes=[ax_x, ax_y], marks=[], title=newplot.title_label)
    newplot = addline(newplot, expr_array[pos])
    return newplot

#place a new plot
def newplot(self):
    global plot_array
    if len(plot_array) < 2 and active_plot > 2:
        plot_array.insert(0, makeplot())
    elif active_plot <= 2:
        if len(plot_array) <= active_plot and len(plot_array) != 0:
            plot_array[active_plot] = makeplot()
        elif len(plot_array) == 0:
            plot_array.append(makeplot())
    else:
        plot_array.insert(0, makeplot())
        plot_array = plot_array[0:3]
    update_viz()

# inserts a new line into the plot
def plotline(self):
    global plot_array
    pos = eq_dropdown.value    
    if len(plot_array) < active_plot:
        plot_array[active_plot] = addline(plot_array[active_plot], \
                                         expr_array[pos])
        update_viz()
    else:
        print('No Graph selected to plot new Line')

# updates plot when changes are done
def updategraph(change):
    global plot_array
    if len(plot_array) <= active_plot and len(plot_array) != 0:
        plot = plot_array[active_plot]
        plot.title_label = title_label_input.value
        plot.x_label = x_label_input.value
        plot.y_labe = y_label_input.value
        try:
            plot.x_steps = int(x_steps_input.value)
        except:
            plot.x_steps = plot.x_steps
        try:
            plot.x_min = int(x_min_input.value)
        except:
            plot.x_min = plot.x_min
        try:
            plot.x_max = int(x_max_input.value)
        except:
            plot.x_max = plot.x_max
        if plot.x_log:
            x_sc = bqp.LogScale(min=plot.x_min, max=plot.x_max)
            x_data = np.geomspace(plot.x_min, plot.x_max, plot.x_steps)
        else:
            x_sc = bqp.LinearScale(min=plot.x_min, max=plot.x_max)
            x_data =  np.linspace(plot.x_min, plot.x_max, plot.x_steps)
        if plot.y_log:
            y_sc = bqp.LogScale(min=plot.x_min, max=plot.x_max)
        else:
            y_sc = bqp.LinearScale(min=plot.x_min, max=plot.x_max)
        ax_x = bqp.Axis(label=plot.x_label, scale=x_sc, grid_lines='solid')
        ax_y = bqp.Axis(label=plot.y_label, scale=y_sc, orientation='vertical', grid_lines='solid')
        lines = []
        for line in enumerate(plot.lines):
            y_data = npfy(x, x_data, line.expr)
            line.x = x_data
            line.y = y_data
            line.scales = {'x': x_sc, 'y': y_sc}
            lines.append(newline)
        fig.axes = [ax_x, ax_y]
        fig.marks = lines
        fig.title = title_label
        plot_array[active_plot] = plot
        update_viz()

<a name="IntElem"></a>
# Interactive Elements 
[Overview](#Overview)

In [30]:
# interactive elements

#text elements
expr_input = widgets.Text(
    value=expr_str,
    description='Expression:',
    disabled=False
)
eq_dropdown = widgets.Dropdown(
    options=[0],
    value=0,
    description='Use:'
)
active_dropdown = widgets.Dropdown(
    options=[0],
    value=0,
    description='Use Graph:'
)

#buttons
send_btn = widgets.Button(
    description='Expr',
    tooltip='Use Equation',
    icon='check'
)
simp_btn = widgets.Button(
    description='simplify',
    tooltip='Simplify Equation',
)
expand_btn = widgets.Button(
    description='Expand',
    tooltip='Expand Equation',
)
setactive_btn = widgets.Button(
    description='Activate',
    tooltip='Activate Equation',
)
plotline_btn = widgets.Button(
    description='Plot',
    tooltip='Plot Line',
)
newplot_btn = widgets.Button(
    description='New Plot',
    tooltip='New Plot',
)
difx_btn = widgets.Button(
    description='dx',
    tooltip='diff x',
)
dify_btn = widgets.Button(
    description='dy',
    tooltip='diff y',
)
difz_btn = widgets.Button(
    description='dz',
    tooltip='diff z',
)
#output
eq_out0 = widgets.Label(value='')
eq_out1 = widgets.Label(value='')
eq_out2 = widgets.Label(value='')

newplot_btn.on_click(newplot)
setactive_btn.on_click(setactive)
plotline_btn.on_click(plotline)
active_dropdown.observe(setactivegraph, 'value')
eqchoice = widgets.HBox([eq_dropdown, active_dropdown, setactive_btn, plotline_btn, newplot_btn])


In [31]:
# interactive elements
title_label_input = widgets.Text(
    value='Title',
    description='Title:'
)
x_label_input = widgets.Text(
    value='X',
    description='X-Label:',
    disabled=False)
y_label_input = widgets.Text(
    value='Y',
    description='Y-Label:'
)
x_steps_input = widgets.Text(
    value='100',
    description='data points:'
)
x_min_input = widgets.Text(
    value='-1',
    description='x min:'
)
x_max_input = widgets.Text(
    value='1',
    description='x max:'
)

title_label_input.observe(updategraph, 'value')
x_label_input.observe(updategraph, 'value')
y_label_input.observe(updategraph, 'value')
x_steps_input.observe(updategraph, 'value')
x_min_input.observe(updategraph, 'value')
x_max_input.observe(updategraph, 'value')

<a name="Layout"></a>
# Layout
[Overview](#Overview)

In [49]:
send_btn.on_click(readexpr)
simp_btn.on_click(simplify)
expand_btn.on_click(expand)
difx_btn.on_click(derix)
input_box = widgets.HBox([expr_input, send_btn, simp_btn, expand_btn, difx_btn])
left_box = widgets.VBox([input_box, eq_out0, eqchoice, history_vbox])
update_gui()

HBox(children=(VBox(children=(HBox(children=(Text(value='sin(x)', description='Expression:'), Button(description='Expr', icon='check', style=ButtonStyle(), tooltip='Use Equation'), Button(description='simplify', style=ButtonStyle(), tooltip='Simplify Equation'), Button(description='Expand', style=ButtonStyle(), tooltip='Expand Equation'), Button(description='dx', style=ButtonStyle(), tooltip='diff x'))), Label(value='\\(\\sin{\\left (x \\right )}\\)'), HBox(children=(Dropdown(description='Use:', options=(0, 1, 2, 3, 4), value=0), Dropdown(description='Use Graph:', options=(0,), value=0), Button(description='Activate', style=ButtonStyle(), tooltip='Activate Equation'), Button(description='Plot', style=ButtonStyle(), tooltip='Plot Line'), Button(description='New Plot', style=ButtonStyle(), tooltip='New Plot'))), VBox())), VBox()))

HBox(children=(VBox(children=(HBox(children=(Text(value='sin(x)', description='Expression:'), Button(description='Expr', icon='check', style=ButtonStyle(), tooltip='Use Equation'), Button(description='simplify', style=ButtonStyle(), tooltip='Simplify Equation'), Button(description='Expand', style=ButtonStyle(), tooltip='Expand Equation'), Button(description='dx', style=ButtonStyle(), tooltip='diff x'))), Label(value='\\(\\sin{\\left (x \\right )}\\)'), HBox(children=(Dropdown(description='Use:', options=(0,), value=0), Dropdown(description='Use Graph:', options=(0,), value=0), Button(description='Activate', style=ButtonStyle(), tooltip='Activate Equation'), Button(description='Plot', style=ButtonStyle(), tooltip='Plot Line'), Button(description='New Plot', style=ButtonStyle(), tooltip='New Plot'))), VBox())), VBox()))

HBox(children=(VBox(children=(HBox(children=(Text(value='sin(x)', description='Expression:'), Button(description='Expr', icon='check', style=ButtonStyle(), tooltip='Use Equation'), Button(description='simplify', style=ButtonStyle(), tooltip='Simplify Equation'), Button(description='Expand', style=ButtonStyle(), tooltip='Expand Equation'), Button(description='dx', style=ButtonStyle(), tooltip='diff x'))), Label(value='\\(\\sin{\\left (x \\right )}\\)'), HBox(children=(Dropdown(description='Use:', options=(0, 1), value=0), Dropdown(description='Use Graph:', options=(0,), value=0), Button(description='Activate', style=ButtonStyle(), tooltip='Activate Equation'), Button(description='Plot', style=ButtonStyle(), tooltip='Plot Line'), Button(description='New Plot', style=ButtonStyle(), tooltip='New Plot'))), VBox())), VBox()))

HBox(children=(VBox(children=(HBox(children=(Text(value='sin(x)', description='Expression:'), Button(description='Expr', icon='check', style=ButtonStyle(), tooltip='Use Equation'), Button(description='simplify', style=ButtonStyle(), tooltip='Simplify Equation'), Button(description='Expand', style=ButtonStyle(), tooltip='Expand Equation'), Button(description='dx', style=ButtonStyle(), tooltip='diff x'))), Label(value='\\(\\sin{\\left (x \\right )}\\)'), HBox(children=(Dropdown(description='Use:', options=(0, 1, 2), value=0), Dropdown(description='Use Graph:', options=(0,), value=0), Button(description='Activate', style=ButtonStyle(), tooltip='Activate Equation'), Button(description='Plot', style=ButtonStyle(), tooltip='Plot Line'), Button(description='New Plot', style=ButtonStyle(), tooltip='New Plot'))), VBox())), VBox()))

HBox(children=(VBox(children=(HBox(children=(Text(value='sin(x)', description='Expression:'), Button(description='Expr', icon='check', style=ButtonStyle(), tooltip='Use Equation'), Button(description='simplify', style=ButtonStyle(), tooltip='Simplify Equation'), Button(description='Expand', style=ButtonStyle(), tooltip='Expand Equation'), Button(description='dx', style=ButtonStyle(), tooltip='diff x'))), Label(value='\\(\\sin{\\left (x \\right )}\\)'), HBox(children=(Dropdown(description='Use:', options=(0, 1, 2, 3), value=0), Dropdown(description='Use Graph:', options=(0,), value=0), Button(description='Activate', style=ButtonStyle(), tooltip='Activate Equation'), Button(description='Plot', style=ButtonStyle(), tooltip='Plot Line'), Button(description='New Plot', style=ButtonStyle(), tooltip='New Plot'))), VBox())), VBox()))

In [7]:
display(history_vbox)

VBox()